In [ ]:
import functools
import getpass
import json
import os
from typing import List, Optional

import pandas as pd

# Settings
os.environ["QUERY_OPENAI"] = "True"  # if set to True, this generates a random result instead of prompting the LLM
os.environ["OPENAI_MODEL"] = "gpt-3.5-turbo-0125"  # the model to use on the OpenAI API
os.environ["OPENAI_N"] = "15"  # the number of answers to generate per prompt
os.environ["OPENAI_TEMPERATURE"] = "1.0"  # the model temperature to use
os.environ["OPENAI_TIMEOUT"] = "60"  # the timeout for OpenAI API calls
os.environ["TEMPLATE_DIR"] = "templates/"
os.environ["SQLITE_PATH"] = "gpt35_results.sqlite3"  # the path to the SQLite database file. Set this to None to disable storage.

In [ ]:
# set the environment variables before loading fm_matcher, otherwise the settings above will be ignored
from fm_matcher.utils import storage
from fm_matcher.utils.models import Attribute, Parameters, Relation, Result
from fm_matcher.utils.prompt_building import PromptDesign, build_prompts
from fm_matcher.utils.prompt_sending import process_prompt_list
from fm_matcher.utils.prompt_postprocessing import postprocess_answers

all_templates = [f"{mode}" for mode in ("oneToOne", "oneToN", "nToOne", "nToN")]
all_modes = [PromptDesign.oneToOne, PromptDesign.oneToN, PromptDesign.nToOne, PromptDesign.nToN]

In [ ]:
benchmark = pd.read_csv("benchmark/ground_truth.csv")
for side in ("source", "target"):
    benchmark[[f"{side}_schema", f"{side}_relation", f"{side}_attribute"]] = benchmark[side].str.split(".", expand=True)
    benchmark[side] = benchmark[side].str.lower()
benchmark["benchmark"] = True

relation_combinations = benchmark[["source_relation", "target_relation"]].drop_duplicates().values

In [ ]:
@functools.cache
def get_description(schema: str, table: str, attribute: Optional[str] = None) -> str:
    if attribute:
        filename = f"{schema}_{table}_{attribute}.txt"
    else:
        filename = f"{schema}_table_{table}.txt"
    filename_filter = lambda f: f.lower() == filename.lower()
    filename = next(filter(filename_filter, os.listdir("schema_documentations")))
    with open(os.path.join("schema_documentations", filename), "r") as desc_file:
        description = desc_file.read()
    return description


@functools.cache
def get_attributes(schema: str, table: str) -> List[str]:
    table_filter = lambda f: f.lower().startswith(f"{schema}_{table}_") and f.endswith(".txt")
    extract_attr_name = lambda f: f[len(f"{schema}_{table}_"):-len(".txt")]
    return [
        extract_attr_name(attr_file)
        for attr_file in filter(table_filter, os.listdir("schema_documentations/"))
    ]


@functools.cache
def get_relation(schema: str, table: str) -> Relation:
    schema, table = schema.lower(), table.lower()
    description = get_description(schema, table)
    attributes = [
        Attribute(
            name=attr_name.capitalize(),
            description=get_description(schema, table, attr_name),
        ) for attr_name in get_attributes(schema, table)
    ]
    return Relation(
        name=table.capitalize(),
        description=description,
        attributes=attributes,
    )


parameters = [
    Parameters(
        source_relation=get_relation("mimic", source),
        target_relation=get_relation("omop", target),
    ) for source, target in relation_combinations
]
parameters = [storage.store_parameters(p) for p in parameters]

In [ ]:
templates = {
    "oneToOne": [
        {
            "role": "user",
            "content": (
                "Act as a schema matcher for relational schemas. Your task is to create semantic matches"
                " that specify how the elements of the source schema and the target schema semantically"
                " correspond to one another. Two attributes semantically match if and only if there exists"
                " an invertible function that maps all values of one attribute to the other. First, I will"
                " input the name of an attribute from the source schema, a description of the attribute,"
                " the name of the relation in belongs to and a description of this relation. After that, I"
                " will input the same information of a single relation and a single attribute from the"
                " target schema."
            )
        },
        {
            "role": "user",
            "content": (
                "The attribute from the source schema is the following:\n"
                "Attribute name: '{{source_attribute.name}}'\n"
                "Attribute description: '{{source_attribute.description}}'\n"
                "Relation name: '{{source_relation.name}}'\n"
                "Relation description: '{{source_relation.description}}'"
            )
        },
        {
            "role": "user",
            "content": (
                "The attribute from the target schema is the following:\n"
                "Relation name: '{{target_relation.name}}'\n"
                "Relation description: '{{target_relation.description}}'\n"
                "Attribute name: '{{target_attribute.name}}'\n"
                "Attribute description: '{{target_attribute.description}}'"
            ),
        },
        {
            "role": "user",
            "content": (
                "Explain whether '{{source_attribute.name}}' from '{{source_relation.name}}' of the source"
                " schema semantically match to '{{target_attribute.name}}' from '{{target_relation.name}}'"
                " of the target schema.. Lets work this out step by step to make sure we get it correct."
                " After your explanation, give a final decision JSON-formatted like this:"
                " `{ \"yes\": [], \"no\": [] }`. Under each of the following keys, list"
                " '{{target_attribute.name}}' if it applies: yes - if there is an invertible"
                " function that maps all values of the source attribute to the values of the target"
                " attribute; no - if there is no such function. Do not mention the attribute if there is"
                " not enough information to decide."
            )
        }
    ],
    "oneToN": [
        {
            "role": "user",
            "content": (
                "Act as a schema matcher for relational schemas. Your task is to create semantic matches"
                " that specify how the elements of the source schema and the target schema semantically"
                " correspond to one another. Two attributes semantically match if and only if there exists"
                " an invertible function that maps all values of one attribute to the other. First, I will"
                " input the name of an attribute from the source schema, a description of the attribute,"
                " the name of the relation in belongs to and a description of this relation. After that, I"
                " will input the same information of a single relation and all its attributes from the"
                " target schema."
            )
        },
        {
            "role": "user",
            "content": (
                "The attribute from the source schema is the following:\n"
                "Attribute name: '{{source_attribute.name}}'\n"
                "Attribute description: '{{source_attribute.description}}'\n"
                "Relation name: '{{source_relation.name}}'\n"
                "Relation description: '{{source_relation.description}}'"
            )
        },
        {
            "role": "user",
            "content": (
                "The relation from the target schema is the following:\n"
                "Relation name: '{{target_relation.name}}'\n"
                "Relation description: '{{target_relation.description}}'\n"
                "In the following, I will list all attributes of '{{target_relation.name}}'."
            )
        },
        {
            "role": "user",
            "content": (
                "Attribute name: '{{target_attribute.name}}'\n"
                "Attribute description: '{{target_attribute.description}}'"
            )
        },
        {
            "role": "user",
            "content": (
                "Explain which of the target attributes semantically match to '{{source_attribute.name}}'"
                " from '{{source_relation.name}}' of the source schema. Lets work this out step by step to"
                " make sure we get it correct. After your explanation, give a final decision"
                " JSON-formatted like this: `{ \"yes\": [], \"no\": [] }`. Under each of the following"
                " keys, list all target attributes of '{{target_relation.name}}' that apply: yes - if"
                " there is an invertible function that maps all values of the source attribute to the"
                " values of the target attribute; no - if there is no such function. Do not mention an"
                " attribute if there is not enough information to decide."
            )
        }
    ],
    "nToOne": [
        {
            "role": "user",
            "content": (
                "Act as a schema matcher for relational schemas. Your task is to create semantic matches"
                " that specify how the elements of the source schema and the target schema semantically"
                " correspond to one another. Two attributes semantically match if and only if there exists"
                " an invertible function that maps all values of one attribute to the other. First, I will"
                " input the name of a single relation from the source schema, the description of the"
                " relation and the name and description of all its attributes. After that, I will input"
                " the same information of a single relation and a single attribute from the target schema."
            ),
        },
        {
            "role": "user",
            "content": (
                "The relation from the source schema is the following:\n"
                "Relation name: '{{source_relation.name}}'\n"
                "Relation description: '{{source_relation.description}}'\n"
                "In the following, I will list all attributes of '{{source_relation.name}}'."
            ),
        },
        {
            "role": "user",
            "content": (
                "Attribute name: '{{source_attribute.name}}'\n"
                "Attribute description: '{{source_attribute.description}}'"
            ),
        },
        {
            "role": "user",
            "content": (
                "The attribute from the target schema is the following:\n"
                "Relation name: '{{target_relation.name}}'\n"
                "Relation description: '{{target_relation.description}}'\n"
                "Attribute name: '{{target_attribute.name}}'\n"
                "Attribute description: '{{target_attribute.description}}'"
            ),
        },
        {
            "role": "user",
            "content": (
                "Explain which of the source attributes semantically match to '{{target_attribute.name}}'"
                " from '{{target_relation.name}}' of the target schema. Lets work this out step by step to"
                " make sure we get it correct. After your explanation, give a final decision"
                " JSON-formatted like this: `{ \"yes\": [], \"no\": [] }`. Under each of the following"
                " keys, list all target attributes of '{{source_relation.name}}' that apply: yes - if"
                " there is an invertible function that maps all values of the source attribute to the"
                " values of the target attribute; no - if there is no such function. Do not mention an"
                " attribute if there is not enough information to decide."
            ),
        }
    ],
    "nToN": [
        {
            "role": "user",
            "content": (
                "Act as a schema matcher for relational schemas. Your task is to create semantic matches"
                " that specify how the elements of the source schema and the target schema semantically"
                " correspond to one another. Two attributes semantically match if and only if there exists"
                " an invertible function that maps all values of one attribute to the other. First, I will"
                " input the name of a single relation from the source schema, the description of the"
                " relation and the name and description of all its attributes. After that, I will input"
                " the same information of one relation and all its attribute from the target schema."
            ),
        },
        {
            "role": "user",
            "content": (
                "The relation from the source schema is the following:\n"
                "Relation name: '{{source_relation.name}}'\n"
                "Relation description: '{{source_relation.description}}'\n"
                "In the following, I will list all attributes of '{{source_relation.name}}'."
            ),
        },
        {
            "role": "user",
            "content": (
                "Attribute name: '{{source_attribute.name}}'\n"
                "Attribute description: '{{source_attribute.description}}'"
            ),
        },
        {
            "role": "user",
            "content": (
                "The relation from the target schema is the following:\n"
                "Relation name: '{{target_relation.name}}'\n"
                "Relation description: '{{target_relation.description}}'\n"
                "In the following, I will list all attributes of '{{target_relation.name}}'."
            ),
        },
        {
            "role": "user",
            "content": (
                "Attribute name: '{{target_attribute.name}}'\n"
                "Attribute description: '{{target_attribute.description}}'"
            ),
        },
        {
            "role": "user",
            "content": (
                "Explain which pairs of attributes from the source and target schema semantically match."
                " Lets work this out step by step to make sure we get it correct. After your explanation,"
                " give a final decision JSON-formatted like this: `{ \"yes\": [], \"no\": [] }`. List all"
                " attribute pairs written as `<source attribute>,<target attribute>` under one the"
                " following keys where they apply: yes - if there is an invertible function that maps all"
                " values of the source attribute to the values of the target attribute; no - if there is"
                " no such function. Do not mention an attribute pair if there is not enough information to"
                " decide."
            ),
        }
    ],
}

os.makedirs(os.getenv("TEMPLATE_DIR"), exist_ok=True)
for tpl_name in all_templates:
    with open(os.path.join(os.getenv("TEMPLATE_DIR"), f"{tpl_name}.json"), "w") as jsn:
        json.dump(templates[tpl_name.split("_")[-1]], jsn)

In [ ]:
prompts = {
    parameter.meta["path"]: build_prompts(parameters=parameter, templates=all_templates, modes=all_modes)
    for parameter in parameters
}

prompts = {
    param_path: [storage.store_prompt(prompt) for prompt in prompt_list]
    for param_path, prompt_list in prompts.items()
}


os.environ["OPENAI_API_KEY"] = getpass.getpass("Your OpenAI API key please:")
results = {}
for parameter in parameters:
    answers = await process_prompt_list(parameter, prompts[parameter.meta["path"]])
    results[parameter.meta["path"]] = storage.store_result(postprocess_answers(parameter, answers))

In [ ]:
def result_to_dataframe(result: Result) -> pd.DataFrame:
    df_data = []
    for attribute_pair, result_pair in result.pairs.items():
        for vote in result_pair.votes:
            left_scope = "1" if len(vote.answer.attributes.sources) == 1 else "n"
            right_scope = "1" if len(vote.answer.attributes.targets) == 1 else "n"
            task_scope = left_scope + "-to-" + right_scope
            source = "mimic." + result.parameters.source_relation.name + "." + attribute_pair.source.name
            target = "omop." + result.parameters.target_relation.name + "." + attribute_pair.target.name
            df_data.append({
                "task_scope": task_scope,
                "source": source.lower(),
                "source_attribute": attribute_pair.source.name,
                "source_relation": result.parameters.source_relation.name,
                "target": target.lower(),
                "target_attribute": attribute_pair.target.name,
                "target_relation": result.parameters.target_relation.name,
                "decision": str(vote.vote),
                "decision_index": vote.answer.index,
            })
    return pd.DataFrame(df_data)


result_df = pd.DataFrame()
for result in results.values():
    result_df = pd.concat((result_df, result_to_dataframe(result)))

result_df = result_df.merge(benchmark[["source", "target", "benchmark"]], on=["source", "target"], how="left").copy()
result_df["benchmark"] = result_df["benchmark"].fillna(False)
result_df.to_csv("results/gpt35_results.csv", index=False)
result_df